# IndicF5 Gradio Demo

This notebook installs the required packages, loads the optimized `app.py`, and launches the Gradio interface.

In [ ]:
# Install dependencies (run once)
%pip install -q transformers==4.41.2 accelerate==0.33.0 f5-tts==0.3.4 vocos==0.1.0 gradio==4.31.0
# You may need to restart the kernel after installation

In [ ]:
# Import the app and launch Gradio
import os, sys
sys.path.append(os.getcwd())  # ensure local imports work

# Reload the updated app.py
import importlib
app_module = importlib.import_module('app')
# The app module defines `iface` at the bottom; launch it
app_module.iface.launch()